In [1]:
import mlflow
import mlflow.sklearn

In [2]:

# Set the tracking URI
mlflow.set_tracking_uri("http://127.0.0.1:5001")


In [76]:
mlflow.end_run()


In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

with mlflow.start_run():
    # Load the Iris dataset
    data = load_iris()
    X = data.data
    y = data.target

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    n_estimators = 50

    # Create and train a RandomForestClassifier
    clf = RandomForestClassifier(n_estimators)
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Log metrics
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    # Log parameters
    mlflow.log_params({"n_estimators": n_estimators, "random_state": 45})

    # Log the model as an artifact
    mlflow.sklearn.log_model(clf, "model")

MlflowException: API request to http://127.0.0.1:5001/api/2.0/mlflow/runs/create failed with exception HTTPConnectionPool(host='127.0.0.1', port=5001): Max retries exceeded with url: /api/2.0/mlflow/runs/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x13f34c910>: Failed to establish a new connection: [Errno 61] Connection refused'))

# Use ML Flow to select model with best Accuracy score:


In [3]:
best_run = mlflow.search_runs(order_by=["metrics.accuracy desc"]).iloc[0]
best_run_id = best_run.run_id
best_model = mlflow.sklearn.load_model("runs:/" + best_run_id + "/model")

# Declare Pydantic Object for API

In [4]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()


class InputData(BaseModel):
    feature1: float
    feature2: float
    feature3: float
    feature4: float

In [5]:
mlflow.sklearn.load_model("runs:/" + best_run_id + "/model")

RandomForestClassifier(n_estimators=50)

In [7]:
@app.post("/predict/")
async def predict(input_data: InputData):
    # Extract input features from the request
    features = [input_data.feature1, input_data.feature2, input_data.feature3, input_data.feature4]

    # Make predictions using the loaded model
    prediction = model.predict([features])[0]

    # Create a response dictionary
    response_data = {"prediction": prediction}

    return response_data


# Problem:
- Needs to be in proper environment to run. ie(fastAPItest.py)

# Example Request:
`curl -X POST "http://localhost:8000/predict/" -H "accept: application/json" -H "Content-Type: application/json" -d '{"feature1": 5.1, "feature2": 3.5, "feature3": 1.4, "feature4": 0.2}'`
